# Replica project

### Pretrained networks

Testing the different pretrained networks and resolutions for best performance.

In [1]:
# loading the metadata
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from tqdm import tqdm
from glob import glob

import torch

sys.path.insert(0, "../model/")
from utils import *
from metrics import *
from model_replica import ReplicaNet


path = '/home/guhennec/scratch/2021_Cini/TopologicalAnalysis_Cini/data/'
data_dir = '/scratch/students/schaerf/'

2022-04-28 11:39:21.756791: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
data = pd.read_csv(data_dir + 'full_data.csv').drop(columns=['Unnamed: 0', 'level_0'])
print(data.shape)
data.head()

(6163, 45)


,uid_connection,index,uid,Drawer,ImageNumber,Author,Description,AuthorOriginal,AuthorULANLabel,AuthorULAN,...,AuthorBirthCity,AuthorDeathCity,CardboardURL,ImageURL,img1,img2,type,annotated,cluster,set
0,002ca9c06a704c779e69d93cb4fadaa3,649,45588b4e33444ed8b34c523bac808bc4,158B,556,TIZIANO Vecellio (copia da),S.Mariá Maddalena. cm. 123 x 91,TIZIANO (copia da -),Titian,ulan:500031075,...,Belluno,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/158B%2F158...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,45588b4e33444ed8b34c523bac808bc4,da54c106180449d1a52993e9a5bc17fd,POSITIVE,2018-07-24 14:58:44.559049+00:00,50.0,train
1,0038ab462149452b98a4d7359a06f9f4,1571,ae828b07e4284d80b652206ffb77278a,158C,120,TIZIANO Vecellio (copia da),"Morte di S. Pietro Martire. mm. 123,8 x 84,2",TIZIANO (copia da -),Titian,ulan:500031075,...,Belluno,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/158C%2F158...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,ae828b07e4284d80b652206ffb77278a,ea64207684694d379579998f88b7c29a,POSITIVE,2018-07-16 09:47:57.987515+00:00,51.0,test
2,00474897de044f7fb1ef16888dc198f3,1107,76b2c4aeab404bc79768117e3ddc62b1,47A,858,BELLINI Giovanni (cerchia di),Pietà.,"BELLINI, Giovanni (cerchia di -)","Bellini, Giovanni",ulan:500019244,...,Venice,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/47A%2F47A_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,76b2c4aeab404bc79768117e3ddc62b1,b8dd2c0c2c7146e68e5bbb0aaf6cf803,POSITIVE,2018-03-14 10:46:45.603341+00:00,498.0,test
3,009c03c265b8418495da038bc3826bf7,1764,c5839e5220e5419e92d878396b0c04bb,135C,57,SEC XVI ex,Un soffitto. Part.: Leda e il cigno.,SEC. XVI ex.,NaN,NaN,...,NaN,NaN,https://dhlabsrv4.epfl.ch/iiif_cini/135C%2F135...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,450ee2aceb014709b23a93e1163937c5,c5839e5220e5419e92d878396b0c04bb,POSITIVE,2018-05-07 08:28:11.797723+00:00,92.0,train
4,00aeddb93a8a4120aa984eda6c822747,747,4d02860f58f2443ba4d1ef088756f854,21C,228,BASSANO Leandro,"La preparazione del pranzo. cm. 88,3 x 102,9","BASSANO, Leandro.","Bassano, Leandro",ulan:500015945,...,Bassano del Grappa,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/21C%2F21C_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,4d02860f58f2443ba4d1ef088756f854,27bad0f213a342c3987e8cec61e40550,POSITIVE,2018-03-23 15:41:53.094226+00:00,218.0,train


### Creating embeddings of all the images

In [3]:
img = preprocess_image(data_dir + 'subset/' + data['uid'][0] + '.jpg')
newmodel = create_model('resnet50', 'max')
embedding = get_embedding(img, newmodel, type='pretrained').squeeze(1).squeeze(1)
print(embedding.shape)
embedding

(2048,)


array([0.01462797, 0.02479225, 0.02289144, ..., 0.01756204, 0.02368654,
       0.02203275], dtype=float32)

In [27]:
newmodel = create_model('resnet50', 'max')
embeddings = [[uid, get_embedding(preprocess_image(data_dir + 'subset/' + uid + '.jpg'), newmodel).squeeze(1).squeeze(1)] for uid in tqdm(data['uid'].unique())]

100%|██████████| 4357/4357 [23:13<00:00,  3.13it/s]


In [ ]:
np.save('resnet50_max_480.npy', np.array(embeddings, dtype=np.ndarray))

In [7]:
embeddings = np.load(data_dir + 'resnet50_max_240.npy', allow_pickle=True)
embeddings

array([['45588b4e33444ed8b34c523bac808bc4',
        array([0.01273958, 0.02077351, 0.02422727, ..., 0.01849872, 0.02054269,
       0.01348173], dtype=float32)],
       ['ae828b07e4284d80b652206ffb77278a',
        array([0.01653748, 0.02765266, 0.02819748, ..., 0.01199384, 0.01694367,
       0.01413397], dtype=float32)],
       ['76b2c4aeab404bc79768117e3ddc62b1',
        array([0.01377614, 0.02740756, 0.02341555, ..., 0.01013662, 0.01947277,
       0.01492655], dtype=float32)],
       ...,
       ['ffd362f3b0f64665a6d4708b62c6333c',
        array([0.01821662, 0.01876744, 0.0185441 , ..., 0.01633258, 0.02021206,
       0.01388135], dtype=float32)],
       ['ffd9ce3649364906810269748b8a5ddc',
        array([0.01877906, 0.0240321 , 0.02262984, ..., 0.01359423, 0.01878336,
       0.01530929], dtype=float32)],
       ['fffb47ce9194475087f2c7dc1eaaa41a',
        array([0.02208385, 0.02518187, 0.02769189, ..., 0.01206282, 0.02445544,
       0.01236099], dtype=float32)]], dtype=object)

### Finding most similar embeddings

In [4]:
list_downloaded = [
    file.split("/")[-1].split(".")[0] for file in glob(data_dir + "subset/*")
]
print(len(list_downloaded))

train_test = data[data["set"].notnull()].reset_index() #[data['img1'].isin(list_downloaded)][data['img2'].isin(list_downloaded)]
print(train_test.shape)


4593
(2862, 46)


In [5]:

for model in ['resnext-101','regnet_x_32gf', 'resnet101', 'resnet152']: #, "efficientnet0", "efficientnet7" 'densenet161',==3 dimensions?? 
    for pool in ["avg"]: #, "max"
        for resolution in ["480"]: #"240", 
            print(model, pool, resolution)
            embeddings = np.load(
                data_dir + 'embeddings/' + model + "_" + pool + "_" + resolution + ".npy",
                allow_pickle=True,
            )        
            embs = get_lower_dimension(embeddings, dimensions=500, method='pca')
            embeddings_new = np.array([[embeddings[i, 0], embs[i]] for i in range(len(embs))], dtype=np.ndarray)
            #embs = get_lower_dimension_(embeddings, dimensions=500, method='svd')
            #embeddings_new_2 = np.array([[embeddings[i, 0], embs[i]] for i in range(len(embs))], dtype=np.ndarray)
            print('original scores')
            get_scores(embeddings, train_test, data, list_downloaded)
            print('dimensionality reduction scores 500')
            get_scores(embeddings_new, train_test, data, list_downloaded)
            print('\n\n\n\n')
            #print('dimensionality reduction scores 250')
            #get_scores(embeddings_new_2, train_test, data, list_downloaded)
            
            

resnext-101 avg 480
original scores


100%|██████████| 2862/2862 [00:17<00:00, 166.11it/s]


all positions 285.85919196748745
min positions 119.34677419354838
median positions 327.06129032258065
mean average precision 0.12330041312737605
recall @ 400 0.5402476958525345
recall @ 200 0.4332667690732207
recall @ 100 0.35131592421915003
recall @ 50 0.26671786994367636
recall @ 20 0.20953661034306195
dimensionality reduction scores 500


100%|██████████| 2862/2862 [00:08<00:00, 352.30it/s]


all positions 286.2098972029644
min positions 123.20806451612903
median positions 327.35403225806454
mean average precision 0.1165710427884686
recall @ 400 0.5349148745519714
recall @ 200 0.42513696876600104
recall @ 100 0.34475230414746544
recall @ 50 0.26742831541218637
recall @ 20 0.20004096262160775





regnet_x_32gf avg 480
original scores


...

all positions 305.3939756155869
min positions 134.88870967741934
median positions 353.7911290322581
mean average precision 0.06819496340542985
recall @ 400 0.4634171786994368
recall @ 200 0.3545673323092678
recall @ 100 0.27870775729646696
recall @ 50 0.21350358422939064
recall @ 20 0.14147977470558115





resnet152 avg 480
original scores


100%|██████████| 2862/2862 [00:16<00:00, 178.78it/s]


all positions 302.4754960554626
min positions 133.6725806451613
median positions 349.4290322580645
mean average precision 0.0930941107878368
recall @ 400 0.4839279313876088
recall @ 200 0.3765988223246288
recall @ 100 0.2831899641577061
recall @ 50 0.22901817716333847
recall @ 20 0.17410394265232976
dimensionality reduction scores 500


100%|██████████| 2862/2862 [00:07<00:00, 368.81it/s]


all positions 303.21969878077937
min positions 133.18548387096774
median positions 352.69354838709677
mean average precision 0.08954008016960034
recall @ 400 0.4749391961085509
recall @ 200 0.3704051459293395
recall @ 100 0.2925902457757297
recall @ 50 0.22731822836661547
recall @ 20 0.17456541218637991







In [7]:
device = 'cuda'
model = ReplicaNet('resnext-101', device)
model.load_state_dict(torch.load(data_dir + "model_weights_" + 'resnext-101'))

<All keys matched successfully>

In [ ]:
embeddings = [[uid, get_embedding(preprocess_image(data_dir + 'subset/' + uid + '.jpg', resolution=360), model, device).squeeze(1).squeeze(1)] for uid in tqdm(data['uid'].unique())]

In [12]:
np.save(data_dir + 'resnext-101_epoch_6.npy', np.array(embeddings, dtype=np.ndarray))

In [4]:
for model in ['resnext-101',]: 
    for pool in ["avg"]: 
        for resolution in ["360"]: 
            for epoch in ['1']:      #, '6'      
                print(model, pool, resolution, 'epoch ', epoch)
                embeddings = np.load(
                    data_dir + 'embeddings/resnext-101_epoch_' + epoch + ".npy",
                    allow_pickle=True,
                )        
                print('original scores')
                get_scores(embeddings, train_test, data, list_downloaded)
            

resnext-101 avg 360 epoch  1
original scores


100%|██████████| 2862/2862 [00:10<00:00, 278.31it/s]


all positions 267.6495338273966
min positions 89.64354838709677
median positions 307.26451612903224
mean average precision 0.17015603007121957
recall @ 400 0.6022113415258576
recall @ 200 0.5179115463389657
recall @ 100 0.4260835893497184
recall @ 50 0.34440028161802355
recall @ 20 0.28553443420378904


In [4]:
embeddings = np.load(
                    data_dir + 'embeddings/resnext-101_avg_480' + ".npy",
                    allow_pickle=True,
                )        
embeddings.shape

get_scores(embeddings, train_test, data, list_downloaded)

100%|██████████| 2862/2862 [00:10<00:00, 264.86it/s]


all positions 285.85919196748745
min positions 119.34677419354838
median positions 327.06129032258065
mean average precision 0.12329775891310613
recall @ 400 0.5398380696364568
recall @ 200 0.4330619559651818
recall @ 100 0.35111111111111115
recall @ 50 0.26671786994367636
recall @ 20 0.20953661034306195


(285.85919196748745,
 119.34677419354838,
 327.06129032258065,
 0.12329775891310613,
 0.5398380696364568,
 0.4330619559651818,
 0.35111111111111115,
 0.26671786994367636,
 0.20953661034306195)

In [5]:
embeddings = np.load(path + 'Replica_UIDs_ResNet_VGG_All.npy',allow_pickle=True)
print(embeddings.shape)
embeddings = embeddings[np.isin(embeddings[:,0], list(data["uid"].unique()))]

embeddings.shape

(329480, 3)


(4357, 3)

In [6]:
embeddings = embeddings[:,:2]#[order]
embeddings.shape

(4357, 2)

In [10]:
np.save(data_dir + 'embeddings/benoit.npy', embeddings)

In [7]:
get_scores(embeddings, train_test, data, list_downloaded, True)

100%|██████████| 2862/2862 [00:23<00:00, 123.57it/s]


all positions 171.7779105904853
min positions 43.262903225806454
median positions 173.16370967741935
mean average precision 0.5398631955614576
recall @ 400 0.8570922939068099
recall @ 200 0.8272791858678954
recall @ 100 0.7868381976446492
recall @ 50 0.7304032258064516
recall @ 20 0.6659901433691756


(171.7779105904853,
 43.262903225806454,
 173.16370967741935,
 0.5398631955614576,
 0.8570922939068099,
 0.8272791858678954,
 0.7868381976446492,
 0.7304032258064516,
 0.6659901433691756)

In [14]:
embeddings = np.load(data_dir + 'embeddings/benoit.npy', allow_pickle=True)        
        
for method in ['pca','umap', 'svd', ]:  #'tsne'
    for dimension in [2, 50, 100, 500]:  
        print(method, dimension)
        embs = get_lower_dimension(embeddings, dimensions=dimension, method=method)
        embeddings_new = np.array([[embeddings[i, 0], embs[i]] for i in range(len(embs))], dtype=np.ndarray)
        print('dimensionality reduction scores')
        get_scores(embeddings_new, train_test, data, list_downloaded, True)


pca 2
dimensionality reduction scores


100%|██████████| 2862/2862 [00:07<00:00, 402.21it/s]


all positions 339.7645230695673
min positions 200.08709677419355
median positions 371.708064516129
mean average precision 0.011677539803727781
recall @ 400 0.4086437532002048
recall @ 200 0.24560739887352792
recall @ 100 0.13100742447516642
recall @ 50 0.0654416282642089
recall @ 20 0.03247759856630825
pca 50
dimensionality reduction scores


100%|██████████| 2862/2862 [00:08<00:00, 341.63it/s]


all positions 178.46904135787713
min positions 47.16935483870968
median positions 181.88951612903224
mean average precision 0.4433886583173794
recall @ 400 0.8401747311827956
recall @ 200 0.8019009216589862
recall @ 100 0.7622958269329237
recall @ 50 0.6883115719406041
recall @ 20 0.6008595750128007
pca 100
dimensionality reduction scores


...

all positions 178.98135309586422
min positions 47.36451612903226
median positions 182.36451612903227
mean average precision 0.43855257885499077
recall @ 400 0.8403859447004608
recall @ 200 0.8002419354838709
recall @ 100 0.7591865079365079
recall @ 50 0.6884805427547364
recall @ 20 0.596334485407066
svd 100
dimensionality reduction scores


100%|██████████| 2862/2862 [00:08<00:00, 324.44it/s]


all positions 175.1403299067655
min positions 45.75645161290323
median positions 177.91612903225806
mean average precision 0.4898631157603849
recall @ 400 0.8491333845366102
recall @ 200 0.8150748847926268
recall @ 100 0.7722779057859703
recall @ 50 0.707083973374296
recall @ 20 0.6319559651817717
svd 500
dimensionality reduction scores


100%|██████████| 2862/2862 [00:11<00:00, 246.74it/s]


all positions 171.13028926607697
min positions 42.94193548387097
median positions 172.95967741935485
mean average precision 0.5409108013014658
recall @ 400 0.8600224014336919
recall @ 200 0.8332123655913979
recall @ 100 0.7845129288274449
recall @ 50 0.7325518433179723
recall @ 20 0.6673559907834101


In [13]:
for method in ['tsne']:  
    for dimension in [2]: #, 50, 100, 500 
        print(method, dimension)
        embs = get_lower_dimension(embeddings, dimensions=dimension, method=method)
        embeddings_new = np.array([[embeddings[i, 0], embs[i]] for i in range(len(embs))], dtype=np.ndarray)
        print('dimensionality reduction scores')
        get_scores(embeddings_new, train_test, data, list_downloaded, True)


tsne 2
dimensionality reduction scores


100%|██████████| 2862/2862 [00:07<00:00, 401.14it/s]


all positions 193.31938799904376
min positions 79.99193548387096
median positions 187.80806451612904
mean average precision 0.5653457160367771
recall @ 400 0.7596441372247824
recall @ 200 0.730515873015873
recall @ 100 0.695999743983615
recall @ 50 0.6833653353814645
recall @ 20 0.6451139272913466


In [10]:
final = make_training_set_orig(embeddings, train_test, data, data_dir, n=2)
final.sample(5)

100%|██████████| 2862/2862 [00:35<00:00, 80.08it/s]

2862
(2792, 4)


,A,B,C,set
2288,fce588c0253e47518e975e5cb555e8a8,1bc3bd59f2e4406ab150ada132ad9171,5a74a04f32e540b19b50f3d0575a96cc,test
2137,5517afb357c340e992c5f7e81c1f63d7,179de993c19543babf3508a668c9329a,884b9275ec7a4b32813c817102a2e9b9,train
1414,ae79feb06e024fca8d9713da23698767,b87900df4bcb46b780a5a06ccafb5447,dcd275a84f824ee6b2e006e8fd0f7dbf,train
1854,5d79b149cf7445a4bfdf23d9b798a68c,066c36f2a35f4ba583e7796a212203f1,2a6bc15814964956a95bb30e7bb4fede,test
294,ba9ea6d7c94741ae839dfa5a3e843dc9,5e11887c56114a93a52ab33a3a59c780,aa7426ea130340f983f3bc696c0c072e,train


In [6]:
## add different dimensionality reduction efforts

In [2]:
embeddings = np.load(
                    #data_dir + 'resnext-101_avg_480' + ".npy",
                    data_dir + 'embeddings/resnext-101_epoch_314-04-2022_23:25:29.npy',
                    allow_pickle=True,
                )        

In [3]:
data = pd.read_csv(data_dir + 'dedup_data_sample.csv').drop(columns=['Unnamed: 0', 'level_0'])
train_test = data[data["set"].notnull()].reset_index() #[data['img1'].isin(list_downloaded)][data['img2'].isin(list_downloaded)]


In [4]:
for method in ['tsne']:  
    for dimension in [2]: #, 50, 100, 500 
        print(method, dimension)
        embs = get_lower_dimension(embeddings, dimensions=dimension, method=method)
        embeddings_new = np.array([[embeddings[i, 0], embs[i]] for i in range(len(embs))], dtype=np.ndarray)
        print('dimensionality reduction scores')
        get_scores(embeddings_new, train_test, data)


tsne 2
dimensionality reduction scores


100%|██████████| 2862/2862 [00:06<00:00, 459.37it/s]


all positions 274.91861418614184
min positions 139.67001434720228
median positions 299.7446197991392
mean average precision 0.32877714548898634
recall @ 400 0.5779998178133042
recall @ 200 0.5264694495684452
recall @ 100 0.484360410830999
recall @ 50 0.4475359249390813
recall @ 20 0.4109397855452375


In [5]:
np.save(
        data_dir
        + "embeddings/"
        + 'resnext-101'
        + "_tsne_2"
        + ".npy",
        embeddings_new,
        )
